# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Обзор данных

Первым делом откроем данные и ознакомимся с первыми 10 строками

In [ ]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()

In [1]:
data = pd.read_csv('/datasets/data.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Глядя на строки уже можно заметить несколько особенностей:

1. Значения в столбце трудового стажа вещественные, зачастую отрицательные, а так же местами слишком большие. Скорее  всего в столбец с данными пробрались ошибки.
2. В столбце `education` можно заметить "разные" из-за регистра значения, стоит проверить текстовые столбцы на неявные дубликаты и привести к одному формату
3. В столбце `purpose` одни и те же цели описаны разными выражениями, для анализа понадобится лемматизация данного столбца

Посмотрим общую информацию о таблице:

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Только в двух столбцах есть пропуски, считываемые питоном, причем одинаковое количество.
Так же нас устраивает тип значений всех столбцов, кроме `days_employed` -- вещественное количество дней трудового стажа невозможно.

**Вывод:**

**Понадобится дальнейшее изучение столбца `days_employed`, проверка таблицы на пропуски и дубликаты, а так же категоризация данных**

## Предобработка данных

### Обработка пропусков

Первым делом посмотрим на уникальные значения столбца `debt` -- так мы будем знать сколько из клиентов имели заолженность по возвратам кредита. Это значимый для анализа фактор, поэтому хорошо бы знать какое относительное количество данных будут "портить" пропуски.

In [3]:
data['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

Всего два значения 1 и 0. Скорее всего 1 -- была задолженность, 0 -- не было. Запомним их соотношение -- всего должников в данных около 8%

Далее посмотрим на уникальные значения каждого столбца

In [4]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В основном все логично и схоже с общей статистикой по России. Выбиваются 2 значения `-1` и `20`: первое невозможно, а второе, хоть и случается, но вряд ли чаще чем семьи с 4 или 5 детьми. Можно предположить, что этими значениями отмечали пропуски, или что это ошибка при заполнении `1` и `2`. Давайте посмотрим какие значенния `debt` принимают эти строки.

In [5]:
data[data['children'] == -1]['debt'].value_counts()

0    46
1     1
Name: debt, dtype: int64

In [6]:
data[data['children'] == 20]['debt'].value_counts()

0    68
1     8
Name: debt, dtype: int64

Соотношение примерно такое же как в данных в целом, к тому же строк с непонятными значениями довольно мало. Это категориальные переменные, приведем их все к -1 -- это значение сразу дает понять, что не несет в себе ничего близкого к реальности.

In [7]:
data['children'] = data['children'].replace(20, -1)
data['children'].value_counts()

 0    14149
 1     4818
 2     2055
 3      330
-1      123
 4       41
 5        9
Name: children, dtype: int64

Отлично, остались только реальные и положительные дети :) Продолжим работу со столбцами

In [8]:
data['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Из странных значений -- 0. Что за груднички собрались взять кредит? Взглянем на них.

In [9]:
data[data['dob_years'] == 0].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,-2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,-1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,-1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
1149,0,-934.654854,0,среднее,1,женат / замужем,0,F,компаньон,0,201852.430096,покупка недвижимости
1175,0,370879.508002,0,среднее,1,женат / замужем,0,F,пенсионер,0,313949.845188,получение дополнительного образования
1386,0,-5043.219890,0,высшее,0,женат / замужем,0,M,госслужащий,0,240523.618071,сделка с автомобилем
1890,0,NaN,0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,NaN,жилье
1898,0,370144.537021,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,127400.268338,на покупку автомобиля


Разное образование, работа, доходы и цели... А что у них с выплатой долгов?

In [10]:
data[data['dob_years'] == 0]['debt'].value_counts()

0    93
1     8
Name: debt, dtype: int64

Снова примерно то же соотношение -- видимо это полностью случайные пропуски. Заменим их на -1.

In [11]:
data['dob_years'] = data['dob_years'].replace(0, -1)
data['dob_years'].value_counts()

 35    617
 40    609
 41    607
 34    603
 38    598
 42    597
 33    581
 39    573
 31    560
 36    555
 44    547
 29    545
 30    540
 48    538
 37    537
 50    514
 43    513
 32    510
 49    508
 28    503
 45    497
 27    493
 56    487
 52    484
 47    480
 54    479
 46    475
 58    461
 57    460
 53    459
 51    448
 59    444
 55    443
 26    408
 60    377
 25    357
 61    355
 62    352
 63    269
 64    265
 24    264
 23    254
 65    194
 22    183
 66    183
 67    167
 21    111
-1     101
 68     99
 69     85
 70     65
 71     58
 20     51
 72     33
 19     14
 73      8
 74      6
 75      1
Name: dob_years, dtype: int64

In [12]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Здесь нет пропусков, только дубликаты. Пока оставим все как есть.

In [13]:
data['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

In [14]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [15]:
data['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

Пока все в порядке

In [16]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Одно странное значение -- заменим его на 'NA' для ясности

In [17]:
data['gender'] = data['gender'].replace('XNA', 'NA')
data['gender'].value_counts()

F     14236
M      7288
NA        1
Name: gender, dtype: int64

In [18]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [19]:
data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Придется поработать с дубликатами, но без пропусков!

А вот в столбцах `days_employed` и `total_income` пропусков, как мы помним, полно:

In [20]:
print(data['days_employed'].isna().sum())
data['total_income'].isna().sum()

2174


2174

Да и данные в days_employed как мы помним странные... Посмотрим на таблицу еще раз:

In [21]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Интересно, пока все у кого отрицательное количество дней стажа -- компаньоны и сотрудники. Давайте посмотрим, сколько их таких всего и что у них за доход?

In [22]:
data[data['days_employed'] < 0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15906 entries, 0 to 21524
Data columns (total 12 columns):
children            15906 non-null int64
days_employed       15906 non-null float64
dob_years           15906 non-null int64
education           15906 non-null object
education_id        15906 non-null int64
family_status       15906 non-null object
family_status_id    15906 non-null int64
gender              15906 non-null object
income_type         15906 non-null object
debt                15906 non-null int64
total_income        15906 non-null float64
purpose             15906 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 1.6+ MB


15906! Получается положительные значения более "аномальны"!

In [23]:
data[data['days_employed'] < 0]['income_type'].value_counts()

сотрудник          10014
компаньон           4577
госслужащий         1312
предприниматель        1
студент                1
в декрете              1
Name: income_type, dtype: int64

In [24]:
data[data['days_employed'] >= 0]['income_type'].value_counts()

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

In [25]:
print(data[data['days_employed'].isna()]['income_type'].value_counts())
data[data['total_income'].isna()]['income_type'].value_counts() #посмотрим как распределен доход для строк у которых пропуски в двух столбцах

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64


сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Пустые значения распределены равномерно -- видимо это ошибки заполнения. Их заполним, когда разберемся, что произошло с остальными данными.

Зато непустые показывают интересную картину: почти все положительные значения -- у пенсионеров, отрицательные -- у тех, кто на пенсию пока не вышел. Как же так вышло? Давайте посмотрим среднее значения для положительного стажа.

In [26]:
data[data['days_employed'] >= 0]['days_employed'].mean()

365004.3099162686

Пенсионеры проработали по 1000 лет! Давайте посмотрим на них

In [27]:
data[data['days_employed'] >= 0].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью
35,0,394021.072184,68,среднее,1,гражданский брак,1,M,пенсионер,0,77805.677436,на проведение свадьбы
50,0,353731.432338,63,среднее,1,женат / замужем,0,F,пенсионер,0,92342.730612,автомобили
56,0,370145.087237,64,среднее,1,вдовец / вдова,2,F,пенсионер,0,149141.043533,образование
71,0,338113.529892,62,среднее,1,женат / замужем,0,F,пенсионер,0,43929.696397,автомобили
78,0,359722.945074,61,высшее,0,женат / замужем,0,M,пенсионер,0,175127.646000,сделка с автомобилем


Никакой закономерности пока не бросается в глаза. Давайте тогда посмотрим, что же у не пенсионеров? Отрицательный стаж -- очень странно, но может, сгруппировав их по возрасту мы увидим закономерность?

In [28]:
data[data['days_employed'] < 0].groupby('dob_years')['days_employed'].mean()

dob_years
-1    -2200.375775
 19    -633.678086
 20    -684.944308
 21    -709.440930
 22    -781.376775
 23    -827.309437
 24   -1026.405485
 25   -1088.406453
 26   -1200.288052
 27   -1358.153479
 28   -1397.672853
 29   -1553.823200
 30   -1696.039355
 31   -1652.717935
 32   -1735.782175
 33   -1868.655183
 34   -1983.724384
 35   -2108.881612
 36   -2272.773915
 37   -2178.934808
 38   -2307.062965
 39   -2406.564360
 40   -2345.284329
 41   -2433.612130
 42   -2793.169884
 43   -2492.648991
 44   -2814.375145
 45   -2791.307088
 46   -2867.112262
 47   -2963.394330
 48   -2898.197374
 49   -3096.942310
 50   -3096.417569
 51   -3050.331333
 52   -3262.730016
 53   -3280.806279
 54   -2979.965858
 55   -3577.539706
 56   -3271.207884
 57   -3436.439512
 58   -3269.127650
 59   -3865.467480
 60   -3789.996829
 61   -4033.667004
 62   -3029.123002
 63   -4385.872648
 64   -4045.360383
 65   -4059.315441
 66   -3899.935741
 67   -3614.078059
 68   -4551.440293
 69   -3775.909777
 7

В основном с годами модуль среднего стажа растет. Может стаж случайно записали со знаком минус? Давайте прикинем: 365 дней в году, выходит 1000 дней -- чуть меньше трех лет стажа, 3650 -- 10 лет. Вроде правдоподобно, но выходит большая часть клиентов не имеет 10 лет стажа. Но среди клиентов далеко не все -- "сотрудники", значимая часть -- "компаньоны", а так же "госслужащие". Может из-за этого стаж так мал?

In [29]:
data_employed = data[data['income_type'] == 'сотрудник']
data_unemployed = data[data['income_type'] == 'компаньон']
data_gos = data[data['income_type'] == 'госслужащий']
data_employed[data_employed['days_employed'] < 0].groupby('dob_years')['days_employed'].mean()

dob_years
-1    -1907.113148
 19    -810.376486
 20    -777.254312
 21    -707.903013
 22    -760.590433
 23    -862.964863
 24   -1043.795200
 25   -1066.464525
 26   -1148.979988
 27   -1393.772026
 28   -1367.667936
 29   -1522.776378
 30   -1644.749953
 31   -1647.525900
 32   -1752.478057
 33   -1883.868442
 34   -2002.640035
 35   -2058.176983
 36   -2121.547324
 37   -2175.431266
 38   -2342.979009
 39   -2417.399665
 40   -2259.786189
 41   -2393.871444
 42   -2690.040740
 43   -2533.553737
 44   -2937.997276
 45   -2736.069751
 46   -2784.016688
 47   -2816.761427
 48   -2792.069282
 49   -2930.574467
 50   -3311.271589
 51   -3007.122001
 52   -3208.468192
 53   -3268.822463
 54   -2744.184105
 55   -3660.896084
 56   -3393.431800
 57   -3216.243419
 58   -3096.614286
 59   -3918.284305
 60   -3948.506447
 61   -3877.419502
 62   -3103.741487
 63   -4213.770041
 64   -3995.522060
 65   -5208.817492
 66   -4062.117423
 67   -4523.325571
 68   -4564.150928
 69   -3212.316192
 7

Это уже больше походит на правду! А что у компаньонов и госслужащих?

In [30]:
data_unemployed[data_unemployed['days_employed'] < 0].groupby('dob_years')['days_employed'].mean()

dob_years
-1    -1788.865798
 19    -525.137538
 20    -511.714732
 21    -651.260075
 22    -830.277392
 23    -755.057502
 24    -935.311845
 25   -1030.711110
 26   -1134.899176
 27   -1216.406777
 28   -1403.828892
 29   -1462.418319
 30   -1650.216462
 31   -1503.948911
 32   -1548.322836
 33   -1757.231438
 34   -1852.848207
 35   -2024.113883
 36   -2252.807542
 37   -1767.134947
 38   -1957.163738
 39   -2126.955267
 40   -2094.129544
 41   -2189.150589
 42   -2591.164138
 43   -2040.824380
 44   -2334.887108
 45   -2381.526321
 46   -2757.230559
 47   -2926.534481
 48   -2777.256187
 49   -2924.807074
 50   -2310.226541
 51   -2623.710893
 52   -2907.625292
 53   -2965.982675
 54   -3077.793099
 55   -2726.164069
 56   -2702.140190
 57   -3042.590548
 58   -3093.714611
 59   -2903.706639
 60   -3201.990044
 61   -4338.516232
 62   -2585.873812
 63   -4325.744342
 64   -4250.146506
 65   -2600.578826
 66   -2943.113111
 67   -2055.368039
 68   -4437.044585
 70   -6298.476459
 7

In [31]:
data_gos[data_gos['days_employed'] < 0].groupby('dob_years')['days_employed'].mean()

dob_years
-1    -5878.760924
 19    -509.969922
 20    -645.671074
 21    -996.074430
 22    -688.880725
 23    -754.702208
 24   -1248.994331
 25   -1546.767090
 26   -1720.762795
 27   -1658.511522
 28   -1617.946413
 29   -2069.672071
 30   -2200.472018
 31   -2179.922272
 32   -2209.350465
 33   -2236.636413
 34   -2278.741445
 35   -2841.374496
 36   -3389.530826
 37   -3392.247218
 38   -3345.208262
 39   -3450.815579
 40   -4178.889245
 41   -3578.643392
 42   -3983.819597
 43   -3896.927449
 44   -3719.556845
 45   -4090.647432
 46   -4042.344483
 47   -4086.566213
 48   -4095.856269
 49   -4681.640583
 50   -4165.464774
 51   -4405.220393
 52   -4843.424537
 53   -4653.008114
 54   -5043.854088
 55   -5157.666709
 56   -4544.644256
 57   -5868.449809
 58   -4801.960239
 59   -6172.126533
 60   -4865.641237
 61   -4195.697927
 62   -3726.286587
 63   -5153.957270
 64   -3332.919288
 65   -4639.130249
 66   -4767.533800
 69   -4903.096948
 70   -6025.506521
 71    -730.874426
 7

У госслужащих -- самый большой стаж, в среднем доходящий до 20 лет, у работающих -- больше чем у неработающих. Теперь я убеждена -- нужно просто взять модуль столбца!

Что же делать делать с пенсионерами? Возможно их стаж случайно умножили на 100, цифры примерно подходят. Но не будем гадать: у не пенсионеров возраст доходит до 75 -- заменим "тысячелетние" стажи на медианный стаж их возраста. Это будет не самое точное значение, но достаточно точное для наших задач. То же самое сделаем для NaN, предварительно заполнив их в этом столбце единицами

In [32]:
data['days_employed'] = data['days_employed'].fillna(1)

In [33]:
empl_days_by_age = data[data['days_employed'] < 0].groupby('dob_years')['days_employed'].median()
for i in range(len(data)):
    if data.loc[i, 'days_employed'] > 0:
        data.loc[i, 'days_employed'] = empl_days_by_age[data.loc[i, 'dob_years']]
data['days_employed'] = data['days_employed'].abs()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Что же делать с пропусками в `total_income`? Как мы помним, значения в этом столбце были пустыми там же где были пустыми `days_employed`, причем у разных типов занятости. Заполним их средним значением в зависимости от типа занятости

In [34]:
income_by_type = data.groupby('income_type')['total_income'].median()
data['total_income'] = data['total_income'].fillna(-1)
for i in range(len(data)):
    if data.loc[i, 'total_income'] < 0:
        data.loc[i, 'total_income'] = income_by_type[data.loc[i, 'income_type']]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод:**

**Явные и неявные пропуски заполнены, ошибки устранены, можно приступать к дальнейшей обработке данных**

### Замена типа данных

Типы данных которые нам не подходят -- в столбцах `days_employed`. Дробное число дней стажа просто не имеет смысла. Заменим его на int -- это отбросит дробную часть. Наши значения порядка 1000, так что отброшенная часть будет составлять менее 1% и не сильно повлияет на данные.

In [35]:
data['days_employed'] = data['days_employed'].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


**Вывод:**

**Теперь все столбцы нужных форматов**

### Обработка дубликатов

Для начала посмотрим, сколько явных дубликатов в таблице

In [36]:
data.duplicated().sum()

54

Избавимся от них

In [37]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

Так-то лучше. Теперь вспомним какие неявные дубликаты мы нашли: они были в столбцах `education`, `income_type` и `purpose`. Чтобы разобраться с последним понадобится лемматизация. А вот с первыми двумя все проще.

In [38]:
data['education'].value_counts()

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Приведем все значения к одному регистру

In [39]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [40]:
data['education_id'].value_counts()

1    15188
0     5251
2      744
3      282
4        6
Name: education_id, dtype: int64

Теперь id соответсвуют определенным значениям, а не десятке разных! Видимо дубликаты были получены из-за человеческого фактора -- разные люди вводят информацию в разном регистре

In [41]:
income_type_number = data['income_type'].value_counts()
income_type_number

сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Можно распределить единичные значения по категориям: студента, безработных и в декрете отправять к компаньонам, а предпринимателей -- к сотрудникам. Но не испортит ли это статистику?

In [42]:
income_by_type #вспомним средние доходы для различных категорий

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        170898.309923
компаньон          202417.461462
пенсионер          137127.465690
предприниматель    499163.144947
сотрудник          161380.260488
студент             98201.625314
Name: total_income, dtype: float64

Конечно значения разнятся, но не в десятки раз. Прикинем на сколько поменяют среднее наши обобщения:

In [43]:
print((income_by_type['безработный']*income_type_number['безработный'] 
        + income_by_type['в декрете']*income_type_number['в декрете']
      + income_by_type['студент']*income_type_number['студент']
      + income_by_type['компаньон']*income_type_number['компаньон'])/income_type_number[['безработный', 'в декрете', 'студент', 'компаньон']].sum())

202339.7746823806


Совсем не сильно! Так же и с сотрудниками -- можно объединять

In [44]:
data['income_type'] = data['income_type'].replace('предприниматель', 'сотрудник')
data['income_type'] = data['income_type'].replace('безработный', 'компаньон')
data['income_type'] = data['income_type'].replace('в декрете', 'компаньон')
data['income_type'] = data['income_type'].replace('студент', 'компаньон')
income_type_number = data['income_type'].value_counts()
income_type_number

сотрудник      11093
компаньон       5084
пенсионер       3837
госслужащий     1457
Name: income_type, dtype: int64

Также для порядка приведем столбец `family_status` к нижнему регистру

In [45]:
data['family_status'] = data['family_status'].str.lower()
data['family_status'].value_counts()

женат / замужем          12344
гражданский брак          4163
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

**Вывод:**

**Явные и неявные дубликаты устранены, данные более читаемы**

### Лемматизация

В данных о целях, на которые берется кредит, очень много разнообразных причин. Посмотрим на них:

In [46]:
data['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Десятки разных значений, анализ по ним может быть не особо наглядным. Можно ли разделить их на какие-то категории?
Если посмотреть на данные, то сразу бросается в глаза, что в целом многие из целей одинаковые, просто сформулированны по-разному.  Например "автомобиль" и "свой автомобиль" и "на покупку подержанного автомобиля". Все эти значения и многие другие по сути одна цель -- покупка автомобиля.
Оценив данные можно выделить 4 основные категории: свадьба, операции с недвижимостью, покупка автомобиля и образование. Потенциально еще можно выделить категорию Строительство и ремонт, но добавим их к недвижимости.

Вручную избавляться от дубликатов как в других столбцах будет затруднительно. Лемматизируем столбец, а потом поменяем разные леммы на название общей категории.
Применим лемматизацию к столбцу:

In [48]:
data['purpose'] = data['purpose'].apply(m.lemmatize)
data['purpose'].value_counts()

[автомобиль, \n]                                          972
[свадьба, \n]                                             793
[на,  , проведение,  , свадьба, \n]                       773
[сыграть,  , свадьба, \n]                                 769
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           662
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 652
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            625
[покупка

Лемматизация прошла успешно! Посмотрим на список и прикинем какие слова указывают о принадлежности к выделенным нами категориям:
1. свадьба: "свадьба"
2. операции с недвижимостью: "недвижимость", "жилье"
3. покупка автомобиля: "автомобиль"
4. образование: "образование"

Возможно мы что-то упустили, но мы заметим это после первой замены, когда уберется основная масса смысловых дубликатов.
Напишем функцию, которая проверяет принадлежность значения к одной из категорий и применим ее к столбцу.

In [49]:
def categorize_purpose(purpose):
    if 'свадьба' in purpose:
        return 'свадьба'
    if 'недвижимость' in purpose or 'жилье' in purpose:
        return 'операции с недвижимостью'
    if 'автомобиль' in purpose:
        return 'автомобиль'
    if 'образование' in purpose:
        return 'образование'
    return purpose
    

data['purpose'] = data['purpose'].apply(categorize_purpose)
data['purpose'].value_counts()

операции с недвижимостью    10814
автомобиль                   4308
образование                  4014
свадьба                      2335
Name: purpose, dtype: int64

Никаких лишних данных не осталось! Теперь цели явно распределены по 4 понятным категориям

**Вывод:**

**Мы привели данные в столбце с целями к единому виду, распределив их по понятным для анализа категориям**

### Категоризация данных

Можно разделить данные на категории многими подходами и способами. Но нам нужно категоризировать их под наши цели, чтобы анализ стал удобнее и нагляднее. Наш анализ будет проходить по 4 столбцам:
1. количество детей
2. семейное положение
3. доход
4. цели кредита

1, 2 и 4 данные у нас уже неплохо категоризированны. Но 1 и 2 можно улучшить. А 3 нужно категоризировать в целом -- в этом столбце у нас тысячи разных значений, о которых мы мало что знаем на данный момент.

Начем с столбца `children`. В нем у нас указано количество детей. Кто выплачивает кредиты чаще, семьи с одним ребенком или многодетные -- это интересный вопрос, *но перед нами стоит другой*: влияет ли *наличие* детей на возврат кредита в срок. Значит мы можем категоризировать данные попросту на три значения: есть дети(**1**), нет детей(**0**), данные отсутствуют(**-1**). Сделаем это.

In [50]:
def childless(quantity):
    if quantity > 0:
        return 1
    if quantity == 0:
        return 0
    if quantity < 0:
        return -1

data['have_children'] = data['children'].apply(childless)
data['have_children'].value_counts()

 0    14107
 1     7241
-1      123
Name: have_children, dtype: int64

Данные по детям для анализа готовы!

Теперь вспомним значения столбца `family_status`:

In [51]:
data['family_status'].value_counts()

женат / замужем          12344
гражданский брак          4163
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

В целом казалось бы можно разделить их на 2 категории: есть партнер или нет. Однако для финансовых вопросов официальность брака имеет значение: в гражданском браке имущество не считается совместно накопленным. Лучше оставим эту категорию как есть, а вот разведенных, вдо_в и незамужних объединим в одну -- вдовство или развод не должны влиять на имущество, а если бы клиент хотель взять кредит вместе с новым партнер_ом он вероятно указал бы гражданский брак.

Для удобства категоризацию произведем по столбцу `family_status_id` и создадим новый столбец со значениям "брак", "гражданский брак", "нет брака"

In [52]:
data['family_status_id'].value_counts()

0    12344
1     4163
4     2810
3     1195
2      959
Name: family_status_id, dtype: int64

In [53]:
def marriage(id):
    if id == 0:
        return 'брак'
    if id == 1:
        return 'гражданский брак'
    if id == 2 or id == 3 or id == 4:
        return 'нет брака'

data['marriage'] = data['family_status_id'].apply(marriage)
data['marriage'].value_counts()

брак                12344
нет брака            4964
гражданский брак     4163
Name: marriage, dtype: int64

Теперь нужно решить как категоризировать информацию о доходе. Посмотрим минимум, максимум, среднее и медианное значения столбца:

In [54]:
print(data['total_income'].min())
print(data['total_income'].max())
print(data['total_income'].mean())
print(data['total_income'].median())

20667.26379327158
2265604.028722744
167422.06386552742
151898.6934377726


Максимальный доход исчитывается миллионами, а минимальный - двадцать тысяч. Зато средний и медианный почти рядом -- 167 и 152 тысячи. Возможно средний скосили повыше несколько миллионеров.
Попробуем придумать логичные категории, а потом проверим сколько клиентов в них войдут:
1. <50к 
2. 50к<150к
3. 150к<500к
4. 500к+

In [55]:
def categorize_income(income):
    if income <= 50000:
        return 'low'
    if 50000 < income <= 150000:
        return 'middle'
    if 150000 < income <= 500000:
        return 'upper_middle'
    if 500000 < income:
        return 'high'
    
data['income_category'] = data['total_income'].apply(categorize_income)
data['income_category'].value_counts()

upper_middle    10688
middle          10189
low               372
high              222
Name: income_category, dtype: int64

Основная масса поместились в катгеории middle и upper_middle. Это логично но не особо показательно. Зато теперь мы знаем что почти все доходы у нас расположенны в диапазоне от 50к до 500к. Получается нам нужно разделить этот диапазон на еще несколько.
Попробуем такую градацию:
1. <100к 
2. 100к<150к
3. 150к<300к
4. 300к<500к
5. 500к+

In [56]:
def categorize_income(income):
    if income <= 100000:
        return 'low'
    if 100000 < income <= 150000:
        return 'lower_middle'
    if 150000 < income <= 300000:
        return 'middle'
    if 300000 < income <= 500000:
        return 'upper_middle'
    if 500000 < income:
        return 'high'
    
data['income_category'] = data['total_income'].apply(categorize_income)
data['income_category'].value_counts()

middle          9427
lower_middle    6098
low             4463
upper_middle    1261
high             222
Name: income_category, dtype: int64

Возможно это неидеальная категоризация, но она уже более равномерная. Она довольно логична -- люди с доходом выше 300к вероятно реже берут кредиты, им будет проще накопить нужную сумму. Оставим такую категоризацию.

**Вывод:**

**Мы произвели категоризацию данных, окончательно подготовив данные к анализу для ответа на поставленные вопросы**

## Шаг 3. Ответьте на вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для ответа на вопрос посчитаем количество задолженностей по кредитам для двух категорий клиентов: с детьми и без(тех, для кого информация отсутствует, учитывать не будем)

In [57]:
children_debt = data[data['have_children'] == 1]['debt'].sum()
childless_debt = data[data['have_children'] == 0]['debt'].sum()
print('Среди', data[data['have_children'] == 1]['debt'].count(), 'клиентов с детьми было', children_debt, 'задолженностей')
print('Среди', data[data['have_children'] == 0]['debt'].count(), 'клиентов без детей было', childless_debt, 'задолженностей')

Среди 7241 клиентов с детьми было 669 задолженностей
Среди 14107 клиентов без детей было 1063 задолженностей


По этим цифрам сложно оценить зависимость. Посчитаем конвертацию!

In [58]:
print('Процент задолженностей для клиентов с детьми:', '{:.2%}'.format(children_debt/data[data['have_children'] == 1]['debt'].count()))
print('Процент задолженностей для клентов без детей:', '{:.2%}'.format(childless_debt/data[data['have_children'] == 0]['debt'].count()))

Процент задолженностей для клиентов с детьми: 9.24%
Процент задолженностей для клентов без детей: 7.54%


Выходит клиенты с детьми возвращают кредиты хуже, чем клиенты без детей. При этом и берут кредиты реже.

**Вывод:**

**Клиенты с детьми возвращают кредиты хуже, чем клиенты без детей. Возможно это потому что при наличии детей чаще возникают непредвиденные крупные траты.
Но порядок значений похож для двух категорий -- около 8%, так что возможно разница незначительна.**

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Вспомним наши категории семейного положения и посчитаем для них количество задолженностей:

In [59]:
print('Среди', data[data['marriage'] == 'брак']['debt'].count(), 'клиентов в браке', data[data['marriage'] == 'брак']['debt'].sum(), 'задолженностей')
print('Среди', data[data['marriage'] == 'гражданский брак']['debt'].count(), 'клиентов в гражднаском браке', data[data['marriage'] == 'гражданский брак']['debt'].sum(), 'задолженностей')
print('Среди', data[data['marriage'] == 'нет брака']['debt'].count(), 'клиентов не в браке', data[data['marriage'] == 'нет брака']['debt'].sum(), 'задолженностей')

Среди 12344 клиентов в браке 931 задолженностей
Среди 4163 клиентов в гражднаском браке 388 задолженностей
Среди 4964 клиентов не в браке 422 задолженностей


И посчитаем конвертацию:

In [60]:
print('Процент задолженностей для клиентов в браке:', 
      '{:.2%}'.format(data[data['marriage'] == 'брак']['debt'].sum()/data[data['marriage'] == 'брак']['debt'].count()))
print('Процент задолженностей для клиентов в гражданском браке:', 
      '{:.2%}'.format(data[data['marriage'] == 'гражданский брак']['debt'].sum()/data[data['marriage'] == 'гражданский брак']['debt'].count()))
print('Процент задолженностей для клиентов не в браке:', 
      '{:.2%}'.format(data[data['marriage'] == 'нет брака']['debt'].sum()/data[data['marriage'] == 'нет брака']['debt'].count()))

Процент задолженностей для клиентов в браке: 7.54%
Процент задолженностей для клиентов в гражданском браке: 9.32%
Процент задолженностей для клиентов не в браке: 8.50%


Не зря мы поделили клиентов в  официальном браке и гражданском браке на разные категории!

**Вывод:**

**Чаще всего задолженности случаются у клиентов в гражданском браке, а реже всего -- в официальном. Это довольно закономерно: пары в официальном браке даже в случае развода чаще всего делят кредиты пополам, долг выплачивают два человека. Клиенты без брака вероятно при взятии кредита рассчитывают только на свои доходы, и им будет сложнее его выплатить. А пары в гражданском браке скорее всего выплачивают кредит вместе, только вот в случае расставания кредит официально будет только у того кто его брал, поэтому легко взять кредит, ожидая поддержку партнера, а потом резко ее лишиться.**

**Однако стоит отметить, что для трех категорий(гражданский/официальный брак, отсутствие брака) процент задолженностей близок по порядку**

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Сразу посчитаем конверсию для разных уровней дохода:

In [61]:
print('Процент задолженностей для клиентов с доходом ниже 100000 рублей в месяц:', 
      '{:.2%}'.format(data[data['income_category'] == 'low']['debt'].sum()/data[data['income_category'] == 'low']['debt'].count()))
print('Процент задолженностей для клиентов с доходом от 100000 до 150000 рублей в месяц:', 
      '{:.2%}'.format(data[data['income_category'] == 'lower_middle']['debt'].sum()/data[data['income_category'] == 'lower_middle']['debt'].count()))
print('Процент задолженностей для клиентов с доходом от 150000 до 300000 рублей в месяц:', 
      '{:.2%}'.format(data[data['income_category'] == 'middle']['debt'].sum()/data[data['income_category'] == 'middle']['debt'].count()))
print('Процент задолженностей для клиентов с доходом от 300000 до 500000 рублей в месяц:', 
      '{:.2%}'.format(data[data['income_category'] == 'upper_middle']['debt'].sum()/data[data['income_category'] == 'upper_middle']['debt'].count()))
print('Процент задолженностей для клиентов с доходом выше 500000 рублей в месяц:', 
      '{:.2%}'.format(data[data['income_category'] == 'high']['debt'].sum()/data[data['income_category'] == 'high']['debt'].count()))


Процент задолженностей для клиентов с доходом ниже 100000 рублей в месяц: 7.93%
Процент задолженностей для клиентов с доходом от 100000 до 150000 рублей в месяц: 8.63%
Процент задолженностей для клиентов с доходом от 150000 до 300000 рублей в месяц: 8.01%
Процент задолженностей для клиентов с доходом от 300000 до 500000 рублей в месяц: 7.30%
Процент задолженностей для клиентов с доходом выше 500000 рублей в месяц: 6.31%


**Вывод:**

**Хуже всех возвращают кредиты клиенты с доходом от 100000 до 150000 рублей в месяц. При этом клиенты с доходом ниже возвращают кредиты так же хорошо как клиенты с доходом до 300000 рублей. При доходе выше 300000 задолженностей меньше, а минимум задолженностей у клиентов с самым высоким доходом -- от 500000 рублей.**

### - Как разные цели кредита влияют на его возврат в срок?

Посчитаем конвертацию в должников клиентов с разными целями кредита:

In [62]:
print('Процент задолженностей для клиентов, которые берут кредит на свадьбу:', 
      '{:.2%}'.format(data[data['purpose'] == 'свадьба']['debt'].sum()/data[data['purpose'] == 'свадьба']['debt'].count()))
print('Процент задолженностей для клиентов, которые берут кредит на операции с недвижимостью:', 
      '{:.2%}'.format(data[data['purpose'] == 'операции с недвижимостью']['debt'].sum()/data[data['purpose'] == 'операции с недвижимостью']['debt'].count()))
print('Процент задолженностей для клиентов, которые берут кредит на автомобили:', 
      '{:.2%}'.format(data[data['purpose'] == 'автомобиль']['debt'].sum()/data[data['purpose'] == 'автомобиль']['debt'].count()))
print('Процент задолженностей для клиентов, которые берут кредит на образование:', 
      '{:.2%}'.format(data[data['purpose'] == 'образование']['debt'].sum()/data[data['purpose'] == 'образование']['debt'].count()))


Процент задолженностей для клиентов, которые берут кредит на свадьбу: 7.97%
Процент задолженностей для клиентов, которые берут кредит на операции с недвижимостью: 7.23%
Процент задолженностей для клиентов, которые берут кредит на автомобили: 9.35%
Процент задолженностей для клиентов, которые берут кредит на образование: 9.22%


**Вывод:**

**Самые надежные заемщики берут кредиты на недвижимость. Клиенты, берущие кредит на свадьбу, возвращают их чуть хуже. А вот клиенты с кредитами на автомобили и образование возвращают кредиты хуже всего: почти каждый десятый имел задолженность**

**Вероятно клиенты с кредитами на недвижимость зачастую не сильно меняют свой бюджет: раньше они платили за съемное жилье, теперь примерно те же деньги уходят на кредит.**
**Кредиты на свадьбу скорее всего выплачиваются парой, или даже несколькими семьями, как и с недвижимостью. А вот кредит на авто и тем более образование может быть индивидуальной покупкой. Возможно это объясняет разницу.**

## Шаг 4. Общий вывод

При выполнении задачи была произведена предобработка данных. Были найдены и заполнены пропуски, исправлены ошибки в данных, удалены дубликаты, таблица была приведена к удобному для анализа вида.

Был проведен анализ для каждого из поставленных вопросов. Были обнаружены закономерности, однако так же нужно учитывать, что процент должников различался не более чем 2-3%